In [39]:
%load_ext autoreload
%autoreload 2
import numpy as np
import cvxpy as cvx
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
from tqdm import tqdm
from functools import partial
import sympy as sp
from scipy.optimize import minimize, Bounds
from utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
# swing-up simulation parameters
n = 8                                # state dimension
m = 2                                # control dimension
s_goal = np.array([8, 10, 0, np.pi, 0, 0, 0, 0])  # desired upright pendulum state
s0 = np.array([0, 0, 0, 0, 0, 0, 0, 0])          # initial downright pendulum state
dt = 0.1                             # discrete time resolution
T = 10.                              # total simulation time

# Dynamics
fd = jax.jit(discretize(cartpole, dt))

# SCP parameters
P = 1e3*np.eye(n)                    # terminal state cost matrix
Q = np.diag([1e-2, 1e-2, 1e-3, 1, 1e-2, 1e-2, 1e-3, 1])  # state cost matrix
R = 1e-3*np.eye(m)                   # control cost matrix
ρ = 1.                               # trust region parameter
ru = 8.                              # control effort bound
tol = 5e-1                           # convergence tolerance
max_iters = 100                      # maximum number of SCP iterations

# Solve the swing-up problem with SCP
t = np.arange(0., T + dt, dt)
N = t.size - 1
s, u = solve_swingup_scp(fd, P, Q, R, N, s_goal, s0, ru, ρ, tol, max_iters)

# Simulate open-loop control
for k in range(N):
    s[k+1] = fd(s[k], u[k])

# Plot state and control trajectories
fig, ax = plt.subplots(1, n + 1, figsize=(15, 3), dpi=150)
plt.subplots_adjust(wspace=0.55)
ylabels = (r'$x(t)$', r'$\theta(t)$', r'$\dot{x}(t)$', r'$\dot{\theta}(t)$',
           r'$u(t)$')
for i in range(n):
    ax[i].plot(t, s[:, i], color='tab:blue')
    ax[i].axhline(s_goal[i], linestyle='--', color='tab:orange')
    ax[i].set_xlabel(r'$t$')
    ax[i].set_ylabel(ylabels[i])
ax[n].plot(t[0:N], u)
ax[n].set_xlabel(r'$t$')
ax[n].set_ylabel(ylabels[n])
plt.savefig('cartpole_swingup_limited_actuation.png',
            bbox_inches='tight')
plt.show()

# Animate the solution
fig, ani = animate_cartpole(t, s[:, 0], s[:, 1])
ani.save('cartpole_scp_swingup.mp4', writer='ffmpeg')
plt.show()

8
2


  0%|                                                   | 0/100 [00:02<?, ?it/s]


RuntimeError: SCP solve failed. Problem status: optimal_inaccurate

In [ ]:
def cartpole(s, u):
    """Compute the cart-pole state derivative."""
    g = 9.807          # gravity (m / s**2)
    mQ = 2.5         # mass (kg)
    l = 1.0           # half-length (m)
    IQ = 1.0         # moment of inertia about the out-of-plane axis (kg * m**2)


    # Pendulum
    mp = mQ*4 # this was 4 before
    L = l*2
    Ip = mp*(L/2)**2

    x, y, θ, ϕ, dx, dy, dθ, dϕ = s
    T1, T2 = u
    
    F = T1 + T2
    M = (T1 - T2) * l
    
    Mass = mQ + mp
    
    ds = jnp.array([
        dx,
        dy,
        dθ,
        dϕ,
        (-F*jnp.cos(θ - ϕ) - mQ*L*dϕ*2) * jnp.sin(ϕ) / Mass,
        (-F*jnp.cos(θ - ϕ) - mQ*L*dϕ*2) * (-jnp.cos(ϕ)) / Mass - g,
        M / IQ,
        F*jnp.sin(θ - ϕ) / (mQ * L)
    ])
    return ds

In [5]:
            xl_dot,
            zl_dot,
            phi_dot,
            th_dot,
            (-F*np.cos(phi - th) - self.q_mass*self.cable_length*th_dot*2) * np.sin(th) / self.Mass,
            (-F*np.cos(phi - th) - self.q_mass*self.cable_length*th_dot*2) * (-np.cos(th)) / self.Mass - self.gravity,
            M / self.Ixx,
            F*np.sin(phi - th) / (self.q_mass * self.cable_length)

In [35]:
def discretize(f, dt):
    """Discretize continuous-time dynamics `f` via Runge-Kutta integration."""

    def integrator(s, u, dt=dt):
        k1 = dt * f(s, u)
        k2 = dt * f(s + k1 / 2, u)
        k3 = dt * f(s + k2 / 2, u)
        k4 = dt * f(s + k3, u)
        return s + (k1 + 2 * k2 + 2 * k3 + k4) / 6

    return integrator

In [36]:
@partial(jax.jit, static_argnums=(0,))
@partial(jax.vmap, in_axes=(None, 0, 0))
def linearize(fd: callable,
              s: jnp.ndarray,
              u: jnp.ndarray):
    """Linearize the function `fd(s,u)` around `(s,u)`."""
    # ####################### PART (b): YOUR CODE BELOW #######################

    # INSTRUCTIONS: Use JAX to linearize `fd` around `(s,u)`.

    # TODO: Replace the four lines below with your code.
    #n, m = s.size, u.size
    A = jax.jacfwd(fd,0)(s,u)
    B = jax.jacfwd(fd,1)(s,u)
    c = fd(s,u) - A@s - B@u

    # ############################# END PART (b) ##############################

    return A, B, c

In [37]:
def scp_iteration(fd: callable, P: np.ndarray, Q: np.ndarray, R: np.ndarray,
                  N: int, s_bar: np.ndarray, u_bar: np.ndarray,
                  s_goal: np.ndarray, s0: np.ndarray,
                  ru: float, ρ: float):
    """Solve a single SCP sub-problem for the quadrotor swing-up problem."""
    A, B, c = linearize(fd, s_bar[:-1], u_bar)
    A, B, c = np.array(A), np.array(B), np.array(c)
    n = Q.shape[0]
    m = R.shape[0]
    s_cvx = cvx.Variable((N + 1, n))
    u_cvx = cvx.Variable((N, m))

    # ####################### PART (c): YOUR CODE BELOW #######################

    # INSTRUCTIONS: Construct and solve the convex sub-problem for SCP.

    # TODO: Replace the two lines below with your code.
    objective = cvx.quad_form(s_cvx[N]-s_goal,P)
    constraints = [s_cvx[0] == s0]
    for k in range(N):
        objective += cvx.quad_form(s_cvx[k]-s_goal,Q) + cvx.quad_form(u_cvx[k],R)
        constraints += [cvx.norm(s_cvx[k] - s_bar[k], "inf") <= ρ, cvx.norm(u_cvx[k] - u_bar[k], "inf") <= ρ]
        constraints += [u_cvx[k] <= ru, u_cvx[k] >= -ru]
        constraints += [s_cvx[k+1] == A[k,:,:]@s_cvx[k] + B[k,:,:]@u_cvx[k] + c[k,:]]
        
    # ############################# END PART (c) ##############################

    prob = cvx.Problem(cvx.Minimize(objective), constraints)
    prob.solve()

    if prob.status != 'optimal':
        raise RuntimeError('SCP solve failed. Problem status: ' + prob.status)

    s = s_cvx.value
    u = u_cvx.value
    obj = prob.objective.value

    return s, u, obj

In [38]:
def solve_swingup_scp(fd: callable,
                      P: np.ndarray,
                      Q: np.ndarray,
                      R: np.ndarray,
                      N: int,
                      s_goal: np.ndarray,
                      s0: np.ndarray,
                      ru: float,
                      ρ: float,
                      tol: float,
                      max_iters: int):

    n = Q.shape[0]  # state dimension
    m = R.shape[0]  # control dimension

    # Initialize nominal trajectories
    u_bar = np.zeros((N, m))
    s_bar = np.zeros((N + 1, n))
    print(n)
    print(m)
    s_bar[0] = s0
    for k in range(N):
        s_bar[k+1] = fd(s_bar[k], u_bar[k])

    # Do SCP until convergence or maximum number of iterations is reached
    converged = False
    obj_prev = np.inf
    for i in (prog_bar := tqdm(range(max_iters))):
        s, u, obj = scp_iteration(fd, P, Q, R, N, s_bar, u_bar, s_goal, s0,
                                  ru, ρ)
        diff_obj = np.abs(obj - obj_prev)
        prog_bar.set_postfix({'objective change': '{:.5f}'.format(diff_obj)})

        if diff_obj < tol:
            converged = True
            print('SCP converged after {} iterations.'.format(i))
            break
        else:
            obj_prev = obj
            np.copyto(s_bar, s)
            np.copyto(u_bar, u)

    if not converged:
        raise RuntimeError('SCP did not converge!')

    return s, u